A = W. H

Solution consists of two steps. First, we fix W and learn H, given A. Next, we fix H and learn W, given A of size M by N. We repeat this procedure iteratively. Fixing one variable and learning the other is popularly known as alternating least squares (ALS), as the problem is reduced to a least squares problem. However, we want to constraint W and H to be non-negative, we us non-linear LS (NNLS) instead of basic least squares.

From answer on SE: https://stackoverflow.com/questions/22767695/python-non-negative-matrix-factorization-that-handles-both-zeros-and-missing-dat

Read: https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1


Difference between ALS and Funk SVD: 

1. The objective function in ALS uses L2 norm regularization while Funk uses L1 norm regularization

Remember that L(p) norm is defined as: 

$$
|| x ||_p = \left(\sum_i |x_i|^p\right)^{1/p}
$$

ALS minimizes two loss functions alternatively; It first holds user matrix fixed and runs gradient descent with item matrix; then it holds item matrix fixed and runs gradient descent with user matrix. Its scalability: ALS can run its gradient descent in parallel across multiple partitions of the underlying training data. 

Note that since the user-interaction matrix has missing values, doing a SVD after imputing missing values with 0 is not 
practical. We have to use SGD to reduce errors or ALS etc. The number of latent factors (features) refer to the 
the number of singular values we choose. It is called latent probably because the dimensionality 
is hidden in the input matrix. 


In [282]:
import numpy as np
import pandas as pd
import scipy as sp
from numpy import linalg as LA

M, N = 5, 5
np.random.seed(11)
A_orig = np.abs(np.random.uniform(low=0.0, high=1.0, size=(M,N)))
print (pd.DataFrame(A_orig).head())

          0         1         2         3         4
0  0.180270  0.019475  0.463219  0.724934  0.420204
1  0.485427  0.012781  0.487372  0.941807  0.850795
2  0.729964  0.108736  0.893904  0.857154  0.165087
3  0.632334  0.020484  0.116737  0.316367  0.157912
4  0.758980  0.818275  0.344624  0.318799  0.111661


In [283]:
A = A_orig.copy()
A[0, 0] = np.NAN
A[3, 1] = np.NAN
A[2, 3] = np.NAN

A_df = pd.DataFrame(A)
print (A_df.head())

          0         1         2         3         4
0       NaN  0.019475  0.463219  0.724934  0.420204
1  0.485427  0.012781  0.487372  0.941807  0.850795
2  0.729964  0.108736  0.893904       NaN  0.165087
3  0.632334       NaN  0.116737  0.316367  0.157912
4  0.758980  0.818275  0.344624  0.318799  0.111661


In [284]:
K = 3
W = np.abs(np.random.uniform(low=0, high=1, size=(M, K)))
H = np.abs(np.random.uniform(low=0, high=1, size=(K, N)))
W = np.divide(W, K*W.max())
H = np.divide(H, K*H.max())

pd.DataFrame(W).head()

,0,1,2
0,0.028362,0.240784,0.202547
1,0.018809,0.162093,0.135701
2,0.286477,0.242515,0.203399
3,0.186615,0.320641,0.333333
4,0.114207,0.081038,0.269064


In [285]:
#def cost(A, W, H):
#    from numpy import linalg
#    WH = np.dot(W, H)
#    A_WH = A-WH
#    return linalg.norm(A_WH, 'fro')

We have to skip NaN values in A, so better use below. 

In [286]:
def cost1(A, W, H):
    from numpy import linalg
    mask = pd.DataFrame(A).notnull().values
    WH = np.dot(W, H)
    WH_mask = WH[mask] # WH_mask has MXN - number of missing values. 
    A_mask = A[mask]
    A_WH_mask = A_mask-WH_mask
    # Since now A_WH_mask is a vector, we use L2 instead of Frobenius norm for matrix
    return linalg.norm(A_WH_mask, 2)

In [287]:
from scipy.optimize import nnls

def ALS(A,W,H): 

    num_iter = 1000
    #num_display_cost = max(int(num_iter/10), 1)

    N = int(W.shape[0])
    M = int(H.shape[0])

    for i in range(num_iter):
        if i%2 ==0:
            # Learn H, given A and W
            for j in range(N):
                mask_rows = pd.Series(A[:,j]).notnull()
                H[:,j] = nnls(W[mask_rows], A[:,j][mask_rows])[0]
        else:
            for j in range(M):
                mask_rows = pd.Series(A[j,:]).notnull()
                W[j,:] = nnls(H.transpose()[mask_rows], A[j,:][mask_rows])[0]
        WH = np.dot(W, H)
        c = cost1(A, W, H)

    return W@H

In [288]:
A_pred = pd.DataFrame(np.dot(W, H))
A_pred.head()
#A_pred.shape

,0,1,2,3,4
0,0.072333,0.100804,0.093581,0.106129,0.105185
1,0.048539,0.067735,0.062950,0.071307,0.070626
2,0.078763,0.136069,0.100865,0.137120,0.112462
3,0.114238,0.162682,0.132408,0.169045,0.161089
4,0.072360,0.074712,0.046680,0.080290,0.092314


In [289]:
A_pred.values[~pd.DataFrame(A).notnull().values]

array([0.07233261, 0.13711983, 0.16268151])

In [290]:
A_orig[~pd.DataFrame(A).notnull().values]

array([0.18026969, 0.85715425, 0.02048361])

In [291]:
pd.DataFrame(W).head()

,0,1,2
0,0.028362,0.240784,0.202547
1,0.018809,0.162093,0.135701
2,0.286477,0.242515,0.203399
3,0.186615,0.320641,0.333333
4,0.114207,0.081038,0.269064


In [292]:
pd.DataFrame(H).head()

,0,1,2,3,4
0,0.023447,0.134237,0.025780,0.117579,0.025912
1,0.106864,0.290885,0.333333,0.291812,0.206840
2,0.226794,0.133084,0.062152,0.160607,0.269798


In [293]:
from scipy.sparse.linalg import svds

M, N = 10, 7
B = np.abs(np.random.uniform(low=0.0, high=1.0, size=(M,N)))

# SVD function

def tensorsvd(input,left,right,D):

    T = np.transpose(input,left+right)
    left_index_list = []
    for i in range(len(left)):
        left_index_list.append(T.shape[i])
    xsize = np.prod(left_index_list) 
    right_index_list = []
    for i in range(len(left),len(left)+len(right)):
        right_index_list.append(T.shape[i])
    ysize = np.prod(right_index_list)
    T = np.reshape(T,(xsize,ysize))
    U, s, V = sp.linalg.svd(T,full_matrices = False)
    if D < len(s):
        s = np.diag(s[:D])
        U = U[:,:D]
        V = V[:D,:]
    else:
        D = len(s)
        s = np.diag(s)
    U = np.reshape(U,left_index_list+[D])
    V = np.reshape(V,[D]+right_index_list)
    return U, s, V

u1, v1, s1 = tensorsvd(B,[0],[1],6)
B1 = (u1@v1)@s1
print (LA.norm(B1-B, 'fro'))


0.11470134083000605


Till now, we have created an ALS procedure to deal with missing enteries. Then we tried SVD
on a random matrix and checked how reducing dimensionality affects accuracy measured
through the Frobenius norm. 

In [294]:
def matrix_factorization(R, K, steps=10000, alpha=0.0002, beta=0.02):
    '''
    R: User-item interaction rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''

    print ("YO")

    N = np.shape(R)[0] # Number of User
    M = np.shape(R)[1] # M: Number of Movies rated
    K = M-1 # Num of Features

    P = np.random.rand(N,K)
    Q = np.random.rand(K,M)


    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        #eR = np.dot(P,Q)
        e = 0

        for i in range(len(R)):
            for j in range(len(R[i])):

                if R[i][j] > 0:
                    e += pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)

                    for k in range(K):
                        e += (beta/2) * (P[i][k]**2 + Q[k][j]**2)

        if step%500 == 1:
            print ("Error is", e)

    return P, Q.T

In [295]:
R = [[5,3.,0,1],[4,0,0,1],[1,1,0,5],[1,0,0,4],[0,1,5,4],[2,1,3,0]]
R = np.array(R)

nP, nQ = matrix_factorization(R, K)
nR = pd.DataFrame(nP@nQ.T)

nR.head()





YO
Error is 91.1080101787653
Error is 25.75021982033413
Error is 10.465339074066101
Error is 2.80391140505128
Error is 1.8271238487584216
Error is 1.6227895019385452
Error is 1.5207030252857259
Error is 1.457239799090988
Error is 1.4155873393716316
Error is 1.3876875735719199
Error is 1.3687553118576938
Error is 1.3557331713246277
Error is 1.3466234152742493
Error is 1.3401151007713017
Error is 1.3353488111519254
Error is 1.3317615250389667
Error is 1.3289836318093078
Error is 1.3267710715524657
Error is 1.3249611015811267
Error is 1.323443736998381


,0,1,2,3
0,4.983387,2.964069,3.498864,1.003255
1,3.979942,2.231065,2.963873,1.000534
2,1.014770,0.971259,5.506440,4.969579
3,0.999233,0.955060,4.519771,3.982523
4,2.139175,1.011548,4.978919,3.995674


In real-world, the rating matrix is very sparse. The error function RMSE is only calculated with the non-null rating. 
The missing entries in the rating matrix would be replaced by the dot product of the factor matrices. 
Therefore, we know what to recommend to the users with the unseen movies based on the prediction.

Matrix factorization is a collaborative filtering method to find the relationship between "items" and "users". 
Latent features, the association between users and movies matrices, are determined to find similarity and make a 
prediction based on both item and user entities. 
The matrix factorization of user and item matrices can be generated when the math 
cost function RMSE is minimized through matrix factorization. 
Gradient descent is a method to minimize the cost function.

There is some bug in ALS implementation since it doesn;t match the other method. #TODO
